In [1]:
import os
import pandas as pd
import numpy as np
import pathlib as pth
import numbers
import re
import sys
import pycountry

In [2]:
TRADE_PATH = pth.Path('input/trade/')

ALL_COUNTRIES = set(f.split('.')[0] for f in os.listdir(TRADE_PATH))- {'Hong Kong', 'Singapore'}

GDP_PATH = pth.Path('input/')

print(f"""{TRADE_PATH=}

{GDP_PATH=}

{ALL_COUNTRIES=}
""")

TRADE_PATH=PosixPath('input/trade')

GDP_PATH=PosixPath('input')

ALL_COUNTRIES={'Germany', 'Greece', 'Portugal', 'BelgiumLuxembourg', 'Spain', 'Finland', 'Canada', 'France', 'Belgium', 'Ireland', 'Denmark', 'Mexico', 'Australia', 'Sweden', 'Korea', 'Brazil', 'Switzerland', 'NewZealand', 'UnitedKingdom', 'UnitedStates', 'Austria', 'NetherlandsThe', 'China', 'Norway', 'Japan', 'Italy'}



In [3]:
def is_valid_year_str(year: str):
    return re.match(r'\d{4}[1-4]?', year)

def is_valid_year(year):
    return is_valid_year_str(year) if isinstance(year, str) else isinstance(year, np.Number)

def set_year_type(df):
    """ Return bool for if it needs padding or not and the actual frame """
    first_year = df['Year'].iloc[0]
    if isinstance(first_year, numbers.Number):
        df.astype({'Year': str})
        df['Year'] = df['Year'].astype(str).apply(lambda y: f'{y[:4]}1')  # It may keep floating point so "truncate them": '1234.0' -> '1234'
        return True, df
    elif isinstance(first_year, str) and is_valid_year_str(first_year):
        df = df.astype({'Year': str})
        df['Year'] = df['Year'].apply(lambda y: y.replace('Q', ''))
        return False, df
    else:
        raise NotImplementedError(f"Year has unexpected format {first_year} and type {type(first_year)}, expected int or quarterly year str (eg 2021Q3)")

In [4]:
def clean_trade(df):
    df = df.T
    df = df.drop(df.index[0])
    df = (df
        .rename(columns=df.iloc[0])
        .drop(df.index[0])
        .rename(columns={np.nan: 'Year'})
    )
    needs_padding, df = set_year_type(df)
    df = df.set_index('Year')
    return {'needs_padding': needs_padding, 'df':df}

def make_trade(path):
    dfs = {'import': {}, 'export': {}}
    for root, _, files in os.walk(path):
        for file in filter(lambda p: p.stem in ALL_COUNTRIES, map(pth.Path, files)):
            path = pth.Path(root)/file
            print(path, path.stem)
            
            for sheet in ('Exports, FOB', 'Imports, CIF'):
                df = pd.read_excel(path, skiprows=[1, 2, 3, 4, 5, 7], header=0, sheet_name=sheet)
                trade = sheet.split()[0].lower()[:-2]
                dfs[trade][path.stem] = clean_trade(df)
                
    return dfs

In [5]:
def clean_gdp(df):
    df = (df
        .drop(columns='Unnamed: 0')
        .T
    )
    df = (df
        .rename(columns=df.iloc[0])
        .drop(df.index[0])
        .rename(columns={'Country': 'Year'})
    )
    needs_padding, df = set_year_type(df)
    df = (df
        .set_index('Year')
        .dropna(how='all')
        .filter(items=ALL_COUNTRIES)
    )
    assert len(df.columns) == len(ALL_COUNTRIES), f'{df.columns} {ALL_COUNTRIES.difference(df.columns)=}'
    return {'needs_padding': needs_padding, 'df':df}

def make_gdp(path):
    dfs = {}
    for root, _, files in os.walk(path):
        for file in filter(lambda p: 'GDP' in p.stem, map(lambda f: pth.Path(root)/f, files)):  # Hack filter, change to read all GDPs
            print(file)
            df = (pd
                .read_excel(file, skiprows=[1, 2, 3, 4, 5], header=0)
                .drop(columns=['Unnamed: 2', 'Unnamed: 3'])
            )
            # print(df)
            # trade = sheet.split()[0].lower()[:-2]
            # print(trade)
            dfs[file.stem.replace('GDP', '').lower()] = clean_gdp(df)
    return dfs

In [6]:
def result(trade_frames, gdp_frames):
    """ Each row has the following sequence:
    Country, Year (four times, one per quarter), ...
    """

    assert sys.version_info.major >= 3 and sys.version_info.minor >= 9, "Solution depends on dicts being ordered"

    result = []
    for country in ALL_COUNTRIES:
        lines = {}
        for trade, trade_dfs in trade_frames.items():
            needs_padding, df = trade_dfs[country].values()
            # Check if data was simple int year of quarterly "1960Q1"
            first_year = df.index[0]
            #needs_padding = isinstance(first_year, numbers.Number)

            for year, trade_to_country in df.iterrows():
                lines.setdefault(year, {}).update(
                    {'Year': year}
                    | {f'Export to {k}' if trade == 'export' else f'Import from {k}': v for k, v in trade_to_country.items()}
                )
   
                # GDPs           # TODO: Propably it would be best to have a ALL_YEARS and just get nan if it doesnt exist
                try:
                    lines[year].update({'Nominal GDP': gdp_frames['nominal']['df'][country].loc[year]})
                except KeyError:
                    lines[year].update({'Nominal GDP': np.nan})
                        
                try:
                    lines[year].update({'Real GDP': gdp_frames['real']['df'][country].loc[year]})
                except KeyError:
                    lines[year].update({'Real GDP': np.nan})       

        for year, data in lines.items():
            result.append({'Country': country, **data})
            if needs_padding:
                for i in range(2, 5):
                    result.append({'Country': country, 'Year': f'{year[:4]}{i}'})

    return pd.DataFrame(result)

In [7]:
def bar(s):
    print(f"{30 * '='} {s}")

def done(s):
    bar(f'{s} Done\n')
    
bar('GDP')
gdp_dfs = make_gdp(GDP_PATH)
done('GDP')

bar('Trade')
trade_dfs = make_trade(TRADE_PATH)
done('Trade')

bar('Creating result')
df = result(trade_dfs, gdp_dfs)
done('Creating result')

bar('To excel')
df.to_excel('result.xlsx')
done('To excel')

============================== GDP
input/RealGDP.xlsx
input/NominalGDP.xlsx
============================== GDP Done

============================== Trade
input/trade/Portugal.xls Portugal
WARNING *** file size (509419) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (509419) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
input/trade/Norway.xls Norway
WARNING *** file size (508973) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (508973) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
input/trade/Canada.xls Canada
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
input/trade/UnitedStates.xls U

In [8]:
def fix_country(c):
    if re.match(r'United \w+', c) or c == 'New Zealand':
        return c.replace(' ', '')
    elif c == 'Netherlands':
        return f'{c}The'
    elif 'Korea' in c:
        return 'Korea'
    else:
        return c
    
def fix_time(t):
    return f'{t[:4]}{t[-1]}' # 1965-Q1 -> 19651

def add_value_to_countries(df, col_name):
    for country in ALL_COUNTRIES:
        sub_df = df[df['Country'].str.contains(country)]
        for _unused_index, (_unused_country, time, value) in sub_df.iterrows():
            index = in_df.loc[(in_df['Country'] == country) & (in_df['Year'] == time)].index
            in_df.loc[index, col_name] = value
            
def country_name_from_alpha_3(a3):
    if entry := pycountry.countries.get(alpha_3=a3):
        return entry.name
    else:
        return None

def filter_countries(df):
    return df[df['Country'].str.contains('|'.join(ALL_COUNTRIES))]

def assert_countries_ok(df):
    assert not (diff := set(df['Country']).difference(ALL_COUNTRIES)), diff

In [9]:
in_df = pd.read_csv('input/Tradegdp.csv', delimiter=';')
in_df = in_df.astype({'Year': str})
set(in_df['Country'])
assert_countries_ok(in_df)
in_df

,IFS_code,Country,Year,totimp,impfrau,impfrbe,impfrbelu,impfrfi,impfrfr,impfrgr,...,exptouk,exptous,exptoch,exptobr,exptomex,exptoeger,impfrko,exptoko,Nominal GDP,Real GDP
0,193,Australia,19601,NaN,8.359999999999999,NaN,28.05,10.89,42.02,153.23,...,555.1,136,23.5,NaN,13.7,3.2,NaN,7,17313.0,67915.0
1,193,Australia,19602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,193,Australia,19603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,193,Australia,19604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,193,Australia,19611,NaN,10.34,NaN,20.13,11.88,30.58,137.5,...,469.4,216.3,161.4,.3,8.9,1.6,.33,3.5,17473.0,70022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6081,111,UnitedStates,20194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6082,111,UnitedStates,20201,2336579.064,11619.714249,20886.066736,NaN,4934.742833,43046.757204,115120.055415,...,59009.599581,NaN,124648.519373,35046.884792,212671.750902,NaN,76019.829034,51218.022866,21060474.3,4747469.3
6083,111,UnitedStates,20202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6084,111,UnitedStates,20203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
employment_df = pd.read_csv('input/Employment.csv', usecols=['Country', 'TIME', 'Value', 'SUBJECT'])
employment_df = employment_df[employment_df['SUBJECT'] == 'LFEMTTTT']
employment_df = employment_df.drop(columns=['SUBJECT'])
employment_df['TIME'] = employment_df['TIME'].apply(fix_time)
employment_df['Country'] = employment_df['Country'].apply(fix_country)
employment_df = filter_countries(employment_df)
assert_countries_ok(employment_df)
employment_df

,Country,TIME,Value
460,Australia,19671,4883.2
461,Australia,19672,4907.3
462,Australia,19673,4945.7
463,Australia,19674,4975.6
464,Australia,19681,5000.6
...,...,...,...
12005,UnitedStates,20213,153225.7
12006,UnitedStates,20214,155177.7
12007,UnitedStates,20221,157784.7
12008,UnitedStates,20222,158214.0


In [11]:
add_value_to_countries(employment_df, 'employment')
in_df

,IFS_code,Country,Year,totimp,impfrau,impfrbe,impfrbelu,impfrfi,impfrfr,impfrgr,...,exptous,exptoch,exptobr,exptomex,exptoeger,impfrko,exptoko,Nominal GDP,Real GDP,employment
0,193,Australia,19601,NaN,8.359999999999999,NaN,28.05,10.89,42.02,153.23,...,136,23.5,NaN,13.7,3.2,NaN,7,17313.0,67915.0,NaN
1,193,Australia,19602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,193,Australia,19603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,193,Australia,19604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,193,Australia,19611,NaN,10.34,NaN,20.13,11.88,30.58,137.5,...,216.3,161.4,.3,8.9,1.6,.33,3.5,17473.0,70022.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6081,111,UnitedStates,20194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,158544.0
6082,111,UnitedStates,20201,2336579.064,11619.714249,20886.066736,NaN,4934.742833,43046.757204,115120.055415,...,NaN,124648.519373,35046.884792,212671.750902,NaN,76019.829034,51218.022866,21060474.3,4747469.3,157706.0
6083,111,UnitedStates,20202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137573.3
6084,111,UnitedStates,20203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,146157.3


In [12]:
prod_df = pd.read_csv('input/production industrielle.csv', usecols=['LOCATION', 'TIME', 'Value'])
prod_df['TIME'] = prod_df['TIME'].apply(fix_time)
prod_df = prod_df.rename(columns={'LOCATION': 'Country'})

prod_df['Country'] = prod_df['Country'].apply(lambda a3: country_name_from_alpha_3(a3))
prod_df = prod_df[prod_df['Country'].notna()]
prod_df['Country'] = prod_df['Country'].apply(fix_country)
prod_df = filter_countries(prod_df)

# assert_countries_ok(prod_df)
# prod_df['Country'] = prod_df['Country'].apply(lambda c: pycountry.countries.get(alpha_3=c).name)
prod_df

,Country,TIME,Value
0,Australia,19743,69.75420
1,Australia,19744,67.15520
2,Australia,19751,60.73904
3,Australia,19752,60.87054
4,Australia,19753,63.70487
...,...,...,...
6291,Brazil,20214,95.42844
6292,Brazil,20221,96.04135
6293,Brazil,20222,97.15602
6294,Brazil,20223,97.01955


In [13]:
add_value_to_countries(prod_df, 'prod_ind')
in_df

,IFS_code,Country,Year,totimp,impfrau,impfrbe,impfrbelu,impfrfi,impfrfr,impfrgr,...,exptoch,exptobr,exptomex,exptoeger,impfrko,exptoko,Nominal GDP,Real GDP,employment,prod_ind
0,193,Australia,19601,NaN,8.359999999999999,NaN,28.05,10.89,42.02,153.23,...,23.5,NaN,13.7,3.2,NaN,7,17313.0,67915.0,NaN,NaN
1,193,Australia,19602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,193,Australia,19603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,193,Australia,19604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,193,Australia,19611,NaN,10.34,NaN,20.13,11.88,30.58,137.5,...,161.4,.3,8.9,1.6,.33,3.5,17473.0,70022.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6081,111,UnitedStates,20194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,158544.0,98.44932
6082,111,UnitedStates,20201,2336579.064,11619.714249,20886.066736,NaN,4934.742833,43046.757204,115120.055415,...,124648.519373,35046.884792,212671.750902,NaN,76019.829034,51218.022866,21060474.3,4747469.3,157706.0,97.29951
6083,111,UnitedStates,20202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137573.3,84.42938
6084,111,UnitedStates,20203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,146157.3,93.93282


In [14]:
in_df.to_excel('trade_gdp_employment_industrialprod.xlsx')